In [1]:
%pip show langchain #Version: 0.1.14

Name: langchainNote: you may need to restart the kernel to use updated packages.

Version: 0.1.14
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\Users\Hani\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Imports and Environment

In [1]:
import os

In [2]:
from langchain.document_loaders import PyPDFDirectoryLoader # type: ignore
from langchain.text_splitter import RecursiveCharacterTextSplitter # type: ignore
from langchain.embeddings import OpenAIEmbeddings # type: ignore
from langchain.llms import OpenAI # type: ignore
from langchain.vectorstores import Pinecone# type: ignore
from langchain.chains import RetrievalQA# type: ignore
from langchain.prompts import PromptTemplate# type: ignore
from langchain.chains.summarize import load_summarize_chain# type: ignore
from langchain_community.chat_models import ChatOpenAI# type: ignore

# Load and Chunks Doucment

In [3]:
loader = PyPDFDirectoryLoader("pdf")

In [4]:
data = loader.load()

In [5]:
data[:5]

[Document(page_content='Download free eBooks of classic literature, books and \nnovels at Planet eBook. Subscribe to our free eBooks blog \nand email newsletter.Crime and Punishment\nBy Fyodor Dostoevsky\n', metadata={'source': 'pdf\\crime-and-punishment.pdf', 'page': 0}),
 Document(page_content='Crime and Punishment \x18Translator’s Preface\nA few words about Dostoevsky himself may help the Eng -\nlish reader to understand his work.\nDostoevsky was the son of a doctor. His parents were \nvery hard- working and deeply religious people, but so poor \nthat they lived with their five children in only two rooms. \nThe father and mother spent their evenings in reading aloud \nto their children, generally from books of a serious charac -\nter.\nThough always sickly and delicate Dostoevsky came out \nthird in the final examination of the Petersburg school of \nEngineering. There he had already begun his first work, \n‘Poor Folk.’\nThis story was published by the poet Nekrassov in his \nreview

In [6]:
print(f"You have {len(data)} pages") 

You have 767 pages


# Preprocessing

In [7]:
from pprint import pprint
print(data[6].page_content)

 Free eBooks at Planet eBook.comChapter I
On an exceptionally hot evening early in July a young 
man came out of the garret in which he lodged in S. 
Place and walked slowly, as though in hesitation, towards 
K. bridge.
He had successfully avoided meeting his landlady on the 
staircase. His garret was under the roof of a high, five-sto -
ried house and was more like a cupboard than a room. The 
landlady who provided him with garret, dinners, and at -
tendance, lived on the floor below, and every time he went 
out he was obliged to pass her kitchen, the door of which 
invariably stood open. And each time he passed, the young 
man had a sick, frightened feeling, which made him scowl 
and feel ashamed. He was hopelessly in debt to his landlady, 
and was afraid of meeting her.
This was not because he was cowardly and abject, quite 
the contrary; but for some time past he had been in an over -
strained irritable condition, verging on hypochondria. He 
had become so completely absorbed in h

`Remove opening part`

In [8]:
data_remove_authorsnote = data[5:]

In [9]:
data_remove_authorsnote[1].page_content

'\x18 Free eBooks at Planet eBook.comChapter I\nOn an exceptionally hot evening early in July a young \nman came out of the garret in which he lodged in S. \nPlace and walked slowly, as though in hesitation, towards \nK. bridge.\nHe had successfully avoided meeting his landlady on the \nstaircase. His garret was under the roof of a high, five-sto -\nried house and was more like a cupboard than a room. The \nlandlady who provided him with garret, dinners, and at -\ntendance, lived on the floor below, and every time he went \nout he was obliged to pass her kitchen, the door of which \ninvariably stood open. And each time he passed, the young \nman had a sick, frightened feeling, which made him scowl \nand feel ashamed. He was hopelessly in debt to his landlady, \nand was afraid of meeting her.\nThis was not because he was cowardly and abject, quite \nthe contrary; but for some time past he had been in an over -\nstrained irritable condition, verging on hypochondria. He \nhad become so co

In [10]:
for string in data_remove_authorsnote:
    string.page_content = string.page_content.replace("Free eBooks at Planet eBook.com", "")


**Crime and Punishment /x18**

In [11]:
print(data_remove_authorsnote[2].page_content)

Crime and Punishment stopped on the stairs, to be forced to listen to her trivial, ir -
relevant gossip, to pestering demands for payment, threats 
and complaints, and to rack his brains for excuses, to pre -
varicate, to lie—no, rather than that, he would creep down 
the stairs like a cat and slip out unseen.
This evening, however, on coming out into the street, he 
became acutely aware of his fears.
‘I want to attempt a thing like that  and am frightened 
by these trifles,’ he thought, with an odd smile. ‘Hm … yes, 
all is in a man’s hands and he lets it all slip from cowardice, 
that’s an axiom. It would be interesting to know what it is 
men are most afraid of. Taking a new step, uttering a new 
word is what they fear most…. But I am talking too much. 
It’s because I chatter that I do nothing. Or perhaps it is that 
I chatter because I do nothing. I’ve learned to chatter this 
last month, lying for days together in my den thinking … 
of Jack the Giant-killer. Why am I going there 

**Crime and Punishment (without /x18)**
- Integers are a non trivial case, cannot just remove them using regex

In [12]:
print(data_remove_authorsnote[4].page_content)

Crime and Punishment 10heavy dray horse, suddenly shouted at him as he drove past: 
‘Hey there, German hatter’ bawling at the top of his voice 
and pointing at him—the young man stopped suddenly 
and clutched tremulously at his hat. It was a tall round hat 
from Zimmerman’s, but completely worn out, rusty with 
age, all torn and bespattered, brimless and bent on one side 
in a most unseemly fashion. Not shame, however, but quite 
another feeling akin to terror had overtaken him.
‘I knew it,’ he muttered in confusion, ‘I thought so! That’s 
the worst of all! Why, a stupid thing like this, the most trivial 
detail might spoil the whole plan. Yes, my hat is too notice -
able…. It looks absurd and that makes it noticeable…. With 
my rags I ought to wear a cap, any sort of old pancake, but 
not this grotesque thing. Nobody wears such a hat, it would 
be noticed a mile off, it would be remembered…. What mat -
ters is that people would remember it, and that would give 
them a clue. For this b

In [13]:
for string in data_remove_authorsnote:
    string.page_content = string.page_content.replace("Crime and Punishment \x18", "")
    if string.page_content.startswith("Crime and Punishment"):
        string.page_content = string.page_content[len("Crime and Punishment"):]  # Remove the prefix

In [14]:
print(data_remove_authorsnote[1].page_content)

 Chapter I
On an exceptionally hot evening early in July a young 
man came out of the garret in which he lodged in S. 
Place and walked slowly, as though in hesitation, towards 
K. bridge.
He had successfully avoided meeting his landlady on the 
staircase. His garret was under the roof of a high, five-sto -
ried house and was more like a cupboard than a room. The 
landlady who provided him with garret, dinners, and at -
tendance, lived on the floor below, and every time he went 
out he was obliged to pass her kitchen, the door of which 
invariably stood open. And each time he passed, the young 
man had a sick, frightened feeling, which made him scowl 
and feel ashamed. He was hopelessly in debt to his landlady, 
and was afraid of meeting her.
This was not because he was cowardly and abject, quite 
the contrary; but for some time past he had been in an over -
strained irritable condition, verging on hypochondria. He 
had become so completely absorbed in himself, and iso -
lated from hi

**Larger chunk size for tasks since summarization require a more holistic view of the text.**

In [15]:
print(len(data_remove_authorsnote[2].page_content), "characters per page")

1669 characters per page


In [16]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 700, chunk_overlap=20, 
#                                                separators=[". ",", ","\n",""] ) 
#split into equal chunks

# default separator list of ["\n\n", "\n", " ", ""] 
#can cause words to be split between chunks

In [17]:
# text_chunks = text_splitter.split_documents(data_remove_authorsnote[:3])

In [18]:
# text_chunks

# Experimenting Approaches for summarization

**We will use only first two chapters for experimentation**

In [38]:
data_remove_authorsnote[38].page_content

'\x18Chapter III\nHe waked up late next day after a broken sleep. But his \nsleep had not refreshed him; he waked up bilious, ir -\nritable, ill-tempered, and looked with hatred at his room. \nIt was a tiny cupboard of a room about six paces in length. \nIt had a poverty-stricken appearance with its dusty yellow \npaper peeling off the walls, and it was so low-pitched that \na man of more than average height was ill at ease in it and \nfelt every moment that he would knock his head against the \nceiling. The furniture was in keeping with the room: there \nwere three old chairs, rather rickety; a painted table in the \ncorner on which lay a few manuscripts and books; the dust \nthat lay thick upon them showed that they had been long \nuntouched. A big clumsy sofa occupied almost the whole \nof one wall and half the floor space of the room; it was once \ncovered with chintz, but was now in rags and served Ras -\nkolnikov as a bed. Often he went to sleep on it, as he was, \nwithout undres

**Notes**
- stuff
- map reduce
- refine
- tree summrisation

In [39]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [40]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0613", openai_api_key=OPENAI_API_KEY)

In [41]:
llm.invoke("how can langsmith help with testing?")

RateLimitError: Error code: 429 - {'error': {'message': 'Your account is not active, please check your billing details on our website.', 'type': 'billing_not_active', 'param': None, 'code': 'billing_not_active'}}

In [52]:
data_sample= data_remove_authorsnote[:38]
data_sample[-1]

Document(page_content='\x18\x18 there! And they’re making the most of it! Yes, they are mak -\ning the most of it! They’ve wept over it and grown used to it. \nMan grows used to everything, the scoundrel!’\nHe sank into thought.\n‘And what if I am wrong,’ he cried suddenly after a mo -\nment’s thought. ‘What if man is not really a scoundrel, man \nin general, I mean, the whole race of mankind—then all the \nrest is prejudice, simply artificial terrors and there are no \nbarriers and it’s all as it should be.’', metadata={'source': 'pdf\\crime-and-punishment.pdf', 'page': 42})

In [53]:
from langchain.chains.llm import LLMChain
prompt_template = """Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

llm_chain = LLMChain(llm=llm, prompt=prompt)

In [54]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="text")

In [55]:
print(stuff_chain.run(data_sample[:1]))

RateLimitError: Error code: 429 - {'error': {'message': 'Your account is not active, please check your billing details on our website.', 'type': 'billing_not_active', 'param': None, 'code': 'billing_not_active'}}

## Approach:  Best Representation Vectors

In [41]:
text = ""
for page in data_sample:
     text += page.page_content

In [ ]:
num_tokens = llm.get_num_tokens(text)

In [43]:
print (f"These two chapters have {num_tokens} tokens in it")

These two chapters have 14999 tokens in it


In [48]:
# Loaders
from langchain.schema import Document

# Splitters
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Model
from langchain.chat_models import ChatOpenAI

# Embedding Support
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# Summarizer we'll use for Map Reduce
from langchain.chains.summarize import load_summarize_chain

# Data Science
import numpy as np
from sklearn.cluster import KMeans

In [61]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\t"], chunk_size=500, chunk_overlap=20)

docs = text_splitter.create_documents([text])

In [62]:
num_documents = len(docs)

print (f"Now 2 chapters split up into {num_documents} documents")

Now 2 chapters split up into 126 documents


In [63]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ.get("OPENAI_API_KEY"))

vectors = embeddings.embed_documents([x.page_content for x in docs])

RateLimitError: Error code: 429 - {'error': {'message': 'Your account is not active, please check your billing details on our website.', 'type': 'billing_not_active', 'param': None, 'code': 'billing_not_active'}}

# Initial Try using Azure OpenAI

**Triggers content filters**

In [37]:
from langchain.chains.summarize import load_summarize_chain
from langchain_community.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="chat",
    model="gpt-35-turbo-16k",
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_type="azure",
    api_version = "2023-03-15-preview"
)

KeyError: 'OPENAI_API_BASE'

In [ ]:
map_prompt_template = """
                      Write a summary of this chunk of text that includes the main points and any important details of the mentioned story.
                      {text}
                      """

map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                      Write a concise summary of the following text delimited by triple backquotes.
                      Return your response in bullet points which covers the key points of the text.
                      ```{text}```
                      BULLET POINT SUMMARY:
                      """

combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text"]
)

In [ ]:
map_reduce_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=combine_prompt,
    return_intermediate_steps=True,
)

Refine method for text summarization with LLMs can pull in more relevant context and may be less lossy than Map Reduce. However, it requires many more calls to the LLM than Stuffing, and these calls are not independent, meaning they cannot be parallelized. Additionally, there is some potential dependency on the ordering of the documents. Latest documents they might become more relevant as this method suffers from recency bias.

In [ ]:
map_reduce_outputs = map_reduce_chain({"input_documents": text_chunks})

c:\Users\HAROON TRADERS\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


ValueError: Azure has not provided the response due to a content filter being triggered

In [ ]:
question_prompt_template = """
                  Please provide a summary of the following text.
                  TEXT: {text}
                  SUMMARY:
                  """

question_prompt = PromptTemplate(
    template=question_prompt_template, input_variables=["text"]
)

refine_prompt_template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
              """

refine_prompt = PromptTemplate(
    template=refine_prompt_template, input_variables=["text"]
)

In [ ]:
refine_chain = load_summarize_chain(
    llm,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
)

In [ ]:
refine_outputs = refine_chain({"input_documents": text_chunks})

BadRequestError: Error code: 400 - {'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400}}

https://atlassc.net/2023/11/18/turning-off-azure-openai-service-s-content-filters